# Lab E.4: Graph Classification

**Module:** E - Graph Neural Networks  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate-Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand graph-level vs node-level tasks
- [ ] Implement graph pooling operations (mean, max, sum)
- [ ] Build a complete graph classification pipeline
- [ ] Train on MUTAG (molecular property prediction)
- [ ] Implement attention-based pooling
- [ ] Compare different pooling strategies

---

## 📚 Prerequisites

- Completed: Lab E.2 (GCN from Scratch)
- Completed: Lab E.3 (Graph Attention Networks)
- Knowledge of: GNN message passing, PyTorch basics

---

## 🌍 Real-World Context

**Graph Classification Applications:**

| Domain | Graph | Classification Task |
|--------|-------|--------------------|
| Drug Discovery | Molecules | Toxic vs Non-toxic, Active vs Inactive |
| Malware Detection | Program call graphs | Malicious vs Benign |
| Social Networks | User interaction graphs | Bot vs Human accounts |
| Protein Analysis | Protein structures | Enzyme function prediction |

**The Challenge:** How do we go from node embeddings to a single prediction for the entire graph?

---

## 🧒 ELI5: What Is Graph Classification?

> **Imagine you're a doctor** looking at different types of bacteria under a microscope:
>
> **Node Classification** (Labs E.2-E.3):
> - "Is THIS specific cell part of the nucleus?"
> - You classify individual parts
>
> **Graph Classification** (This Lab):
> - "Is THIS ENTIRE bacterium dangerous?"
> - You classify the whole organism based on its structure
>
> **The Problem:** Each bacterium has a different number of cells (nodes)!
> - Bacterium A: 50 cells
> - Bacterium B: 200 cells
> - How do we compare them with a fixed-size neural network?
>
> **The Solution: Pooling!**
> - "Summarize" all cells into one fixed-size description
> - Like taking an "average" or "maximum" across all cells
> - Then classify that summary

---

## Part 1: Setup and Data Loading

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.utils import to_networkx
import networkx as nx
from sklearn.model_selection import train_test_split
import time

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Load MUTAG dataset
# MUTAG contains 188 molecules labeled by mutagenicity
dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG')

print("=" * 50)
print("MUTAG DATASET")
print("=" * 50)
print(f"Number of graphs: {len(dataset)}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of node features: {dataset.num_node_features}")
print(f"Number of edge features: {dataset.num_edge_features}")

In [ ]:
# Explore dataset statistics
num_nodes = [data.num_nodes for data in dataset]
num_edges = [data.num_edges for data in dataset]

print(f"\n📊 GRAPH STATISTICS")
print("-" * 40)
print(f"Nodes per graph: min={min(num_nodes)}, max={max(num_nodes)}, avg={np.mean(num_nodes):.1f}")
print(f"Edges per graph: min={min(num_edges)}, max={max(num_edges)}, avg={np.mean(num_edges):.1f}")

# Class distribution
labels = [data.y.item() for data in dataset]
print(f"\n📊 CLASS DISTRIBUTION")
print("-" * 40)
for c in range(dataset.num_classes):
    count = labels.count(c)
    print(f"Class {c}: {count} graphs ({100*count/len(dataset):.1f}%)")

In [ ]:
# Visualize a few molecules
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

# Atom type to color mapping (for MUTAG)
atom_colors = ['lightblue', 'orange', 'lightgreen', 'pink', 'yellow', 'cyan', 'magenta']

for i, ax in enumerate(axes):
    data = dataset[i]
    G = to_networkx(data, to_undirected=True)
    
    # Get node colors from features
    node_colors = [atom_colors[data.x[n].argmax().item() % len(atom_colors)] 
                   for n in range(data.num_nodes)]
    
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, ax=ax, node_color=node_colors, 
            node_size=300, with_labels=False, edge_color='gray')
    
    label = "Mutagenic" if data.y.item() == 1 else "Non-mutagenic"
    ax.set_title(f"Molecule {i}\n{label}\n({data.num_nodes} atoms)")

plt.suptitle("Sample Molecules from MUTAG Dataset", fontsize=14)
plt.tight_layout()
plt.show()

print("\n💡 Each molecule is a graph where:")
print("   - Nodes = Atoms")
print("   - Edges = Chemical bonds")
print("   - Task = Predict if the molecule causes genetic mutations")

---

## Part 2: Understanding Graph Pooling

### 2.1 The Pooling Problem

After GNN layers, we have node embeddings:
- Graph 1: 17 nodes → [17, hidden_dim]
- Graph 2: 23 nodes → [23, hidden_dim]
- Graph 3: 11 nodes → [11, hidden_dim]

We need a **fixed-size** representation for classification. This is where **pooling** comes in!

In [ ]:
# Demonstrate the pooling concept

def explain_pooling():
    print("🔄 GRAPH POOLING EXPLAINED")
    print("=" * 50)
    
    # Example: 3 nodes with 4-dim embeddings
    node_embeddings = torch.tensor([
        [1.0, 2.0, 0.5, 3.0],  # Node 0
        [0.5, 1.0, 2.0, 1.0],  # Node 1
        [2.0, 0.0, 1.5, 2.0],  # Node 2
    ])
    
    print("Node embeddings (3 nodes × 4 dimensions):")
    for i, emb in enumerate(node_embeddings):
        print(f"  Node {i}: {emb.tolist()}")
    
    print("\n📊 POOLING OPERATIONS:")
    print("-" * 40)
    
    # Mean pooling
    mean_pool = node_embeddings.mean(dim=0)
    print(f"MEAN pooling: {mean_pool.tolist()}")
    print("  → Average of all nodes. Smooth representation.")
    
    # Max pooling
    max_pool = node_embeddings.max(dim=0)[0]
    print(f"\nMAX pooling: {max_pool.tolist()}")
    print("  → Maximum across nodes. Captures extreme features.")
    
    # Sum pooling
    sum_pool = node_embeddings.sum(dim=0)
    print(f"\nSUM pooling: {sum_pool.tolist()}")
    print("  → Sum of all nodes. Sensitive to graph size.")
    
    print("\n💡 Different pooling captures different aspects!")
    print("   Often, we combine multiple pooling methods.")

explain_pooling()

### 2.2 Batching in PyG

How does PyG handle multiple graphs of different sizes in a batch?

**The `batch` tensor:** Maps each node to its graph index.

In [ ]:
# Create a DataLoader to see batching in action
loader = DataLoader(dataset[:4], batch_size=4, shuffle=False)
batch = next(iter(loader))

print("📦 BATCHING IN PyG")
print("=" * 50)
print(f"\nBatch object: {batch}")
print(f"\nTotal nodes in batch: {batch.num_nodes}")
print(f"Total edges in batch: {batch.num_edges}")

print(f"\n🔑 THE BATCH TENSOR")
print("-" * 40)
print(f"batch.batch shape: {batch.batch.shape}")
print(f"batch.batch values: {batch.batch.tolist()}")

# Count nodes per graph
for i in range(4):
    n_nodes = (batch.batch == i).sum().item()
    print(f"Graph {i}: {n_nodes} nodes")

print("\n💡 The batch tensor tells us which graph each node belongs to!")
print("   This is crucial for pooling - we pool WITHIN each graph.")

In [ ]:
# Demonstrate how global pooling works with batch
print("\n🔄 GLOBAL POOLING WITH BATCH")
print("=" * 50)

# Fake node embeddings (all ones for simplicity)
fake_embeddings = torch.ones(batch.num_nodes, 4)

print(f"Input: {batch.num_nodes} nodes × 4 dims")

# Global mean pooling
pooled = global_mean_pool(fake_embeddings, batch.batch)
print(f"\nAfter global_mean_pool: {pooled.shape}")
print(f"Output: 4 graphs × 4 dims")
print(f"\nPooled values: {pooled}")

print("\n💡 Each graph's nodes are averaged into ONE vector!")

---

## Part 3: Building a Graph Classifier

### 3.1 Basic Architecture

```
Input Graph → GNN Layers → Pooling → MLP Classifier → Class Prediction
```

In [ ]:
class GraphClassifier(nn.Module):
    """
    Graph Neural Network for graph-level classification.
    
    Architecture:
        GCN → ReLU → GCN → ReLU → GCN → Pooling → MLP → Output
    
    Args:
        num_features: Number of input node features
        hidden_dim: Hidden layer dimension
        num_classes: Number of output classes
        pooling: Pooling method ('mean', 'max', 'sum', 'mean_max')
    """
    
    def __init__(self, num_features: int, hidden_dim: int, num_classes: int,
                 pooling: str = 'mean'):
        super().__init__()
        
        self.pooling = pooling
        
        # GNN layers
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        
        # Classifier MLP
        pool_dim = hidden_dim * 2 if pooling == 'mean_max' else hidden_dim
        self.classifier = nn.Sequential(
            nn.Linear(pool_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x, edge_index, batch):
        """
        Forward pass.
        
        Args:
            x: Node features [num_nodes, num_features]
            edge_index: Edge indices [2, num_edges]
            batch: Batch assignment [num_nodes]
        
        Returns:
            Class logits [batch_size, num_classes]
        """
        # GNN layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        
        # Pooling
        if self.pooling == 'mean':
            x = global_mean_pool(x, batch)
        elif self.pooling == 'max':
            x = global_max_pool(x, batch)
        elif self.pooling == 'sum':
            x = global_add_pool(x, batch)
        elif self.pooling == 'mean_max':
            x_mean = global_mean_pool(x, batch)
            x_max = global_max_pool(x, batch)
            x = torch.cat([x_mean, x_max], dim=1)
        else:
            raise ValueError(f"Unknown pooling: {self.pooling}")
        
        # Classify
        return self.classifier(x)
    
    def get_graph_embedding(self, x, edge_index, batch):
        """Get graph embedding before classification."""
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return global_mean_pool(x, batch)

In [ ]:
# Test the model
model = GraphClassifier(
    num_features=dataset.num_node_features,
    hidden_dim=64,
    num_classes=dataset.num_classes,
    pooling='mean_max'
).to(device)

print("Graph Classifier Architecture:")
print("=" * 50)
print(model)

# Test forward pass
batch = next(iter(DataLoader(dataset[:4], batch_size=4))).to(device)
out = model(batch.x, batch.edge_index, batch.batch)

print(f"\nTest batch: 4 graphs")
print(f"Output shape: {out.shape}")
print(f"Output (logits): {out}")

# Parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {total_params:,}")

---

## Part 4: Training the Graph Classifier

In [ ]:
# Create train/test split
dataset = dataset.shuffle()

train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

print(f"Training graphs: {len(train_dataset)}")
print(f"Test graphs: {len(test_dataset)}")

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
def train(model, loader, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = F.cross_entropy(out, batch.y)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * batch.num_graphs
    
    return total_loss / len(loader.dataset)


@torch.no_grad()
def evaluate(model, loader, device):
    """Evaluate model accuracy."""
    model.eval()
    correct = 0
    total = 0
    
    for batch in loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        pred = out.argmax(dim=1)
        correct += (pred == batch.y).sum().item()
        total += batch.num_graphs
    
    return correct / total

In [ ]:
# Training
model = GraphClassifier(
    num_features=dataset.num_node_features,
    hidden_dim=64,
    num_classes=dataset.num_classes,
    pooling='mean_max'
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

history = {'loss': [], 'train_acc': [], 'test_acc': []}

print("Training Graph Classifier on MUTAG...")
print("=" * 60)

best_test_acc = 0

for epoch in range(100):
    loss = train(model, train_loader, optimizer, device)
    train_acc = evaluate(model, train_loader, device)
    test_acc = evaluate(model, test_loader, device)
    
    history['loss'].append(loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch
    
    if epoch % 10 == 0 or epoch == 99:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
              f"Train: {train_acc:.4f} | Test: {test_acc:.4f}")

print("\n" + "=" * 60)
print(f"🎉 Best test accuracy: {best_test_acc:.4f} at epoch {best_epoch}")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['loss'], color='steelblue', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train', linewidth=2)
axes[1].plot(history['test_acc'], label='Test', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Classification Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 5: Comparing Pooling Strategies

In [ ]:
def train_with_pooling(pooling_type, num_runs=5):
    """Train model with specific pooling and return average accuracy."""
    accuracies = []
    
    for run in range(num_runs):
        # Shuffle dataset
        perm = torch.randperm(len(dataset))
        shuffled = dataset[perm]
        
        train_size = int(0.8 * len(dataset))
        train_data = shuffled[:train_size]
        test_data = shuffled[train_size:]
        
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=32)
        
        model = GraphClassifier(
            num_features=dataset.num_node_features,
            hidden_dim=64,
            num_classes=dataset.num_classes,
            pooling=pooling_type
        ).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        
        best_acc = 0
        for epoch in range(100):
            train(model, train_loader, optimizer, device)
            acc = evaluate(model, test_loader, device)
            if acc > best_acc:
                best_acc = acc
        
        accuracies.append(best_acc)
    
    return np.mean(accuracies), np.std(accuracies)

# Compare pooling strategies
print("Comparing Pooling Strategies (5 runs each)...")
print("=" * 50)

pooling_types = ['mean', 'max', 'sum', 'mean_max']
results = {}

for pooling in pooling_types:
    mean_acc, std_acc = train_with_pooling(pooling)
    results[pooling] = (mean_acc, std_acc)
    print(f"{pooling:10s}: {mean_acc:.4f} ± {std_acc:.4f}")

print("\n" + "=" * 50)
best_pooling = max(results, key=lambda x: results[x][0])
print(f"🏆 Best pooling: {best_pooling} ({results[best_pooling][0]:.4f})")

In [ ]:
# Visualize pooling comparison
pooling_names = list(results.keys())
means = [results[p][0] for p in pooling_names]
stds = [results[p][1] for p in pooling_names]

plt.figure(figsize=(10, 6))
bars = plt.bar(pooling_names, means, yerr=stds, capsize=5, 
               color=['steelblue', 'coral', 'seagreen', 'gold'],
               edgecolor='black')
plt.ylabel('Test Accuracy')
plt.title('Pooling Strategy Comparison on MUTAG')
plt.ylim(0.5, 1.0)

# Add value labels
for bar, mean, std in zip(bars, means, stds):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.02,
             f'{mean:.3f}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()

### 🔍 Analysis: Which Pooling Works Best?

- **Mean pooling**: Captures the "average" property of nodes. Good for properties that depend on overall composition.

- **Max pooling**: Captures the "most extreme" feature. Good when a single atom/feature determines the property.

- **Sum pooling**: Sensitive to graph size. Larger molecules naturally have larger representations.

- **Mean + Max**: Combines both - often the best choice in practice!

---

## Part 6: Attention-Based Pooling

Instead of simple pooling, we can **learn** which nodes are important!

In [ ]:
class AttentionPooling(nn.Module):
    """
    Attention-based graph pooling.
    
    Learns to weight nodes by importance:
        attention_score = softmax(MLP(node_embedding))
        graph_embedding = sum(attention_score * node_embedding)
    
    Args:
        hidden_dim: Node embedding dimension
    """
    
    def __init__(self, hidden_dim: int):
        super().__init__()
        
        # Attention network
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, x: torch.Tensor, batch: torch.Tensor):
        """
        Args:
            x: Node embeddings [num_nodes, hidden_dim]
            batch: Batch assignment [num_nodes]
        
        Returns:
            Graph embeddings [batch_size, hidden_dim]
            Attention weights [num_nodes] (for visualization)
        """
        # Compute attention scores
        attn_scores = self.attention(x).squeeze(-1)  # [num_nodes]
        
        # Softmax within each graph
        from torch_geometric.utils import softmax
        attn_weights = softmax(attn_scores, batch)  # [num_nodes]
        
        # Weighted sum
        weighted = x * attn_weights.unsqueeze(-1)  # [num_nodes, hidden_dim]
        out = global_add_pool(weighted, batch)  # [batch_size, hidden_dim]
        
        return out, attn_weights

In [ ]:
class GraphClassifierWithAttention(nn.Module):
    """
    Graph classifier with attention-based pooling.
    """
    
    def __init__(self, num_features: int, hidden_dim: int, num_classes: int):
        super().__init__()
        
        # GNN layers
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        
        # Attention pooling
        self.pool = AttentionPooling(hidden_dim)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x, edge_index, batch, return_attention=False):
        # GNN
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        
        # Attention pooling
        x, attn = self.pool(x, batch)
        
        # Classify
        out = self.classifier(x)
        
        if return_attention:
            return out, attn
        return out

In [ ]:
# Train with attention pooling
def train_attention(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = F.cross_entropy(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    return total_loss / len(loader.dataset)

@torch.no_grad()
def evaluate_attention(model, loader, device):
    model.eval()
    correct = 0
    for batch in loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        pred = out.argmax(dim=1)
        correct += (pred == batch.y).sum().item()
    return correct / len(loader.dataset)

# Train
model_attn = GraphClassifierWithAttention(
    num_features=dataset.num_node_features,
    hidden_dim=64,
    num_classes=dataset.num_classes
).to(device)

optimizer = torch.optim.Adam(model_attn.parameters(), lr=0.01)

print("Training with Attention Pooling...")
print("=" * 50)

best_acc = 0
for epoch in range(100):
    loss = train_attention(model_attn, train_loader, optimizer, device)
    test_acc = evaluate_attention(model_attn, test_loader, device)
    
    if test_acc > best_acc:
        best_acc = test_acc
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Test: {test_acc:.4f}")

print(f"\n🎉 Best test accuracy with attention: {best_acc:.4f}")

In [ ]:
# Visualize attention weights on a molecule
@torch.no_grad()
def visualize_molecule_attention(model, data, idx=0):
    """
    Visualize which atoms the model pays attention to.
    """
    model.eval()
    
    # Get single graph
    graph = data[idx].to(device)
    batch = torch.zeros(graph.num_nodes, dtype=torch.long, device=device)
    
    # Get prediction and attention
    out, attn = model(graph.x, graph.edge_index, batch, return_attention=True)
    pred = out.argmax().item()
    attn = attn.cpu().numpy()
    
    # Normalize attention for visualization
    attn_norm = (attn - attn.min()) / (attn.max() - attn.min() + 1e-8)
    
    # Create graph
    G = to_networkx(graph.cpu(), to_undirected=True)
    pos = nx.spring_layout(G, seed=42)
    
    # Draw
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Original molecule
    atom_colors = ['lightblue', 'orange', 'lightgreen', 'pink', 'yellow', 'cyan', 'magenta']
    node_colors = [atom_colors[graph.x[n].argmax().item() % len(atom_colors)] 
                   for n in range(graph.num_nodes)]
    nx.draw(G, pos, ax=axes[0], node_color=node_colors,
            node_size=500, with_labels=True, edge_color='gray')
    axes[0].set_title(f"Molecule (Ground Truth: {'Mutagenic' if graph.y.item() == 1 else 'Non-mutagenic'})")
    
    # Attention visualization
    node_colors_attn = plt.cm.Reds(attn_norm)
    nx.draw(G, pos, ax=axes[1], node_color=attn_norm, cmap=plt.cm.Reds,
            node_size=500 + 500 * attn_norm, with_labels=True, edge_color='gray')
    axes[1].set_title(f"Attention Weights (Prediction: {'Mutagenic' if pred == 1 else 'Non-mutagenic'})")
    
    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(0, 1))
    sm.set_array([])
    plt.colorbar(sm, ax=axes[1], label='Attention Weight')
    
    plt.tight_layout()
    plt.show()
    
    # Print top-attended atoms
    print(f"\n📊 Top-3 attended atoms:")
    top_atoms = np.argsort(attn)[-3:][::-1]
    for i, atom in enumerate(top_atoms):
        print(f"  {i+1}. Atom {atom}: attention = {attn[atom]:.4f}")

# Visualize a few molecules
for idx in [0, 5, 10]:
    visualize_molecule_attention(model_attn, test_dataset, idx)

---

## ✋ Try It Yourself: Exercise 1

**Task:** Test on a different molecular dataset - PROTEINS.

1. Load the PROTEINS dataset using `TUDataset(root='/tmp/PROTEINS', name='PROTEINS')`
2. Train your graph classifier on it
3. Compare pooling strategies
4. Which works best for protein classification?

In [ ]:
# Your code here!

# proteins = TUDataset(root='/tmp/PROTEINS', name='PROTEINS')
# print(f"PROTEINS: {len(proteins)} graphs, {proteins.num_classes} classes")

# Train and compare...

---

## ✋ Try It Yourself: Exercise 2

**Task:** Implement hierarchical pooling (TopK pooling).

Instead of pooling all nodes at once, TopK pooling:
1. Scores each node
2. Keeps only the top-k% highest scoring nodes
3. Creates a coarsened graph
4. Repeats until we have one "super-node"

Use PyG's `TopKPooling` layer!

In [ ]:
# Your code here!
from torch_geometric.nn import TopKPooling

# Hint:
# pool = TopKPooling(hidden_dim, ratio=0.5)
# x, edge_index, _, batch, _, _ = pool(x, edge_index, batch=batch)

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting the batch tensor
```python
# ❌ Wrong: Pooling without batch (pools ALL nodes)
x = global_mean_pool(x, None)

# ✅ Right: Use batch to pool within each graph
x = global_mean_pool(x, batch)
```
**Why:** Without batch, you get ONE output for ALL graphs!

### Mistake 2: Wrong loss function for imbalanced data
```python
# ❌ Risky: Standard cross-entropy with imbalanced classes
loss = F.cross_entropy(out, y)

# ✅ Better: Use class weights
weights = torch.tensor([1.0, 2.0])  # More weight to minority class
loss = F.cross_entropy(out, y, weight=weights)
```

### Mistake 3: Data leakage in graph datasets
```python
# ❌ Wrong: Shuffling after split (some test graphs in train)
train = dataset[:100]
test = dataset[100:]
train = train.shuffle()

# ✅ Right: Shuffle first, then split
dataset = dataset.shuffle()
train = dataset[:100]
test = dataset[100:]
```

### Mistake 4: Not moving batch to device
```python
# ❌ Wrong: Batch still on CPU
for batch in loader:
    out = model(batch.x, batch.edge_index, batch.batch)  # Error!

# ✅ Right: Move entire batch to device
for batch in loader:
    batch = batch.to(device)
    out = model(batch.x, batch.edge_index, batch.batch)
```

---

## 🎉 Checkpoint

You've learned:
- ✅ The difference between node and graph classification
- ✅ How graph pooling works (mean, max, sum)
- ✅ How PyG batches multiple graphs (batch tensor)
- ✅ How to build end-to-end graph classifiers
- ✅ Attention-based pooling for interpretability
- ✅ Which atoms matter for molecular property prediction!

---

## 🚀 Challenge (Optional)

**Advanced Challenge:** Implement DiffPool (Differentiable Pooling).

DiffPool learns a "soft assignment" of nodes to clusters, then pools within clusters:

1. GNN computes node embeddings
2. Another GNN computes cluster assignments (soft)
3. Nodes are aggregated into clusters
4. Repeat for hierarchical coarsening

This is the state-of-the-art for graph classification!

In [ ]:
# Advanced Challenge: Simplified DiffPool

class SimpleDiffPool(nn.Module):
    """
    Simplified DiffPool layer.
    
    Learns soft cluster assignments.
    """
    
    def __init__(self, in_channels, num_clusters):
        super().__init__()
        # Your code here!
        # Hint: Use a GNN to predict cluster assignments
        pass
    
    def forward(self, x, edge_index, batch):
        # Your code here!
        pass

---

## 📖 Further Reading

- [How Powerful are GNNs?](https://arxiv.org/abs/1810.00826) - WL test and GIN
- [DiffPool Paper](https://arxiv.org/abs/1806.08804) - Hierarchical pooling
- [Graph Classification Survey](https://arxiv.org/abs/2006.16904) - Comprehensive overview
- [MoleculeNet](https://moleculenet.org/) - More molecular datasets
- [OGB Leaderboard](https://ogb.stanford.edu/) - Graph ML benchmarks

---

## 🧹 Cleanup

In [ ]:
# Clear GPU memory
import gc

del model, model_attn

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory after cleanup: {torch.cuda.memory_allocated() / 1e6:.1f} MB")

print("✅ Cleanup complete!")

---

## 🎓 Module E Complete!

Congratulations! You've completed the Graph Neural Networks module!

**What you've mastered:**
- ✅ Graph data structures and PyTorch Geometric
- ✅ Message passing neural networks (GCN)
- ✅ Attention mechanisms for graphs (GAT)
- ✅ Graph-level classification with pooling

**Where to go from here:**
1. **Drug Discovery**: Apply GNNs to molecular property prediction
2. **Knowledge Graphs**: Combine with RAG for structured retrieval
3. **Social Network Analysis**: Fraud detection, recommendation
4. **Research**: Explore GraphTransformers, DiffPool, GIN

**Remember:** GNNs are powerful when your data has **relational structure**. Not everything is a graph - but when it is, GNNs shine!

---

## 🌟 Final Tips

> **Professor SPARK's GNN Wisdom:**
>
> 1. **Start simple**: GCN often works surprisingly well!
> 2. **2-3 layers is enough**: More layers → over-smoothing
> 3. **Pool wisely**: mean+max often beats fancy methods
> 4. **Attention is interpretable**: Use it to explain predictions
> 5. **Scale matters**: For large graphs, use sampling (GraphSAGE)

Happy graph learning! 📊🔗🧠